# RNN/LSTM/GRU

## Bidirectional

paddlepaddle

In [ ]:
def bigru_layer(embedding):
    fc_forward = fluid.layers.fc(input=embedding, size=hidden_dim * 3, bias_attr=False)
    forward = fluid.layers.dynamic_gru(input=fc_forward,
                                       size=hidden_dim,
                                       param_attr=fluid.ParamAttr(name='gru_forward_encoder'))

    fc_backward = fluid.layers.fc(input=embedding, size=hidden_dim * 3, bias_attr=False)
    backward = fluid.layers.dynamic_gru(input=fc_backward,
                                        size=hidden_dim,
                                        param_attr=fluid.ParamAttr(name='gru_backward_encoder'),
                                        is_reverse=True)

    return forward, backward

dialogue_forward, dialogue_backward = bigru_layer(dialogue_embedding)
encoded_dialogue_vector = fluid.layers.concat(input=[dialogue_forward, dialogue_backward], axis=1)

tensorflow2.0

In [39]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        
    def call(self, x, hidden):
        x = self.embedding(x)
        hidden = tf.split(hidden, num_or_size_splits=2, axis=1)
        output, forward_state, backward_state = self.bigru(x, initial_state=hidden)
        state = tf.concat([forward_state, backward_state], axis=1)
        return state 

IndentationError: expected an indented block (<ipython-input-39-6281b5839f8c>, line 4)

In [1]:
import tensorflow as tf

In [4]:
a = tf.ones([2, 5, 10])

In [5]:
a

<tf.Tensor: shape=(2, 5, 10), dtype=float32, numpy=
array([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]], dtype=float32)>

## Deep

paddlepaddle

In [ ]:
paddle.fluid.layers.stack(x, axis=0)

tensorflow2.0

In [6]:
cells = [
    keras.layers.LSTMCell(output_dim),
    keras.layers.LSTMCell(output_dim),
    keras.layers.LSTMCell(output_dim),
]

inputs = keras.Input((timesteps, input_dim))
x = keras.layers.RNN(cells)(inputs)

NameError: name 'keras' is not defined

# LSTM/GRU

paddlepaddle

In [ ]:
paddle.fluid.layers.lstm(input,
                         init_h,
                         init_c,
                         max_len,
                         hidden_size,
                         num_layers,
                         dropout_prob=0.0,
                         is_bidirec=False,
                         is_test=False,
                         name=None,
                         default_initializer=None,
                         seed=-1)

In [ ]:
rnn_out, last_h, last_c = layers.lstm(emb,
                                      init_h,
                                      init_c,
                                      max_len,
                                      hidden_size,
                                      num_layers,
                                      dropout_prob=dropout_prob)

In [ ]:
hidden = fluid.layers.dynamic_gru(input=x, size=hidden_dim)

tensorflow2.0

In [11]:
lstm= tf.keras.layers.LSTM(200)

In [13]:
a, b, c = lstm(hidden, x, cell)

NameError: name 'hidden' is not defined

# Encoder-Decoder结构

paddlepaddle

In [ ]:
import paddle.fluid as fluid
import numpy as np

class MyLayer(fluid.dygraph.Layer):
    def __init__(self, name_scope):
        super(MyLayer, self).__init__(name_scope)
        self.fc = fluid.dygraph.nn.FC(self.full_name(), size=12)
    
    def forward(self, inputs):
        x = self.fc(inputs)
        x = fluid.layers.relu(x)
        self._x_for_debug = x
        x = fluid.layers.elementwise_mul(x, x)
        x = fluid.layers.reduce_sum(x)
        return [x]

tensorflow2.0

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        
    def call(self, x, hidden):
        x = self.embedding(x)
        # hidden = tf.split(hidden, num_or_size_splits=2, axis=1)
        # output, forward_state, backward_state = self.bigru(x, initial_state=hidden)
        # state = tf.concat([forward_state, backward_state], axis=1)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

# 模型Layer、Model构建

##  一些tensor操作

In [14]:
import tensorflow as tf

tf.concat

In [22]:
t1 = [[1, 2, 3], [4, 5, 6]] # 2, 3
t2 = [[7, 8, 9], [10, 11, 12]] 
tf.concat([t1, t2], axis=0) 

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

tf.expand_dims

In [23]:
t3 = [[1, 2, 3],[4, 5, 6]] # shape [2, 3] 

In [26]:
tf.shape(t3)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 3], dtype=int32)>

In [28]:
tf.expand_dims(t3, axis=2) 

<tf.Tensor: shape=(2, 3, 1), dtype=int32, numpy=
array([[[1],
        [2],
        [3]],

       [[4],
        [5],
        [6]]], dtype=int32)>

In [ ]:
tf.expand_dims(t3, 1) 

In [ ]:
tf.expand_dims(t3, 2) 

tf.squeeze

In [29]:
t4 = tf.expand_dims(t3, 2) 

In [30]:
t4

<tf.Tensor: shape=(2, 3, 1), dtype=int32, numpy=
array([[[1],
        [2],
        [3]],

       [[4],
        [5],
        [6]]], dtype=int32)>

In [31]:
tf.squeeze(t4, 2)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

tf.reshape

In [32]:
tf.cast

<function tensorflow.python.ops.math_ops.cast(x, dtype, name=None)>

In [33]:
x = tf.constant([1.8, 2.2], dtype=tf.float32)
tf.dtypes.cast(x, tf.int32) 
# mask = [True , False] loss.astype

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

tf.stack

In [34]:
x = tf.constant([1, 4]) 
y = tf.constant([2, 5]) 
z = tf.constant([3, 6]) 

In [35]:
tf.stack([x, y, z], axis=0) 

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 4],
       [2, 5],
       [3, 6]], dtype=int32)>

In [36]:
tf.stack([x, y, z], axis=1) 

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

## Layer

Encoder

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, embedding_matrix):
       
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

Attention

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, dec_hidden, enc_output):
        

Decoder

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, embedding_matrix):
        super(Decoder, self).__init__()
       

    def call(self, x, context_vector):
        # enc_output shape == (batch_size, max_length, hidden_size)
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        # output shape == (batch_size, vocab)
        out = self.fc(output)
        return x, out, state

## Model

In [ ]:
class SEQ2SEQ(tf.keras.Model):
    def __init__
    
    def call(self, enc_output, dec_hidden, enc_inp, dec_inp):
        predictions = []
        attentions = []

        
 
        return tf.stack(predictions, 1) [batchsize, 20,30000], dec_hidden, attentions

# loss function

In [37]:
cce = tf.keras.losses.CategoricalCrossentropy()
loss = cce(
  [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]], # real
  [[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]]) # prediction
print('Loss: ', loss.numpy()) 

Loss:  0.09458993


In [ ]:
sce = tf.keras.losses.SparseCategoricalCrossentropy()
loss = sce(
  [5, 2, 3, 6, 8], # shape=(3,) real index
  [[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
print('Loss: ', loss.numpy()) 

优化器optimizer

In [ ]:
# 优化器调用api
optimizer = tf.keras.optimizers.Adagrad(params['learning_rate'],
                                        initial_accumulator_value=params['adagrad_init_acc'],
                                        clipnorm=params['max_grad_norm'])

loss函数

In [ ]:
# loss调用api
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
# Whether y_pred is expected to be a logits tensor. 

使用了交叉熵api的LOSS函数

In [ ]:
# 定义loss函数
[2, 4, 5, 6, 7, 1, 1, 1, 1, 1]
def loss_function(real, pred):
    # 判断logit为1和0的数量
    mask = tf.math.logical_not(tf.math.equal(real, 1))
    # 计算decoder的长度
    dec_lens = tf.reduce_sum(tf.cast(mask, dtype=tf.float32), axis=-1)
    # 计算loss值
    loss_ = loss_object(real, pred)
    # 转换mask的格式
    mask = tf.cast(mask, dtype=loss_.dtype)
    # 调整loss
    loss_ *= mask
    # 确认下是否有空的摘要别加入计算
    loss_ = tf.reduce_sum(loss_, axis=-1) / dec_lens
    return tf.reduce_mean(loss_)

自己diy的LOSS函数

In [ ]:
def pgn_log_loss_function(real, final_dists, padding_mask):
    # This is fiddly; we use tf.gather_nd to pick out the probabilities of the gold target words
    loss_per_step = []  # will be list length max_dec_steps containing shape (batch_size)
    batch_nums = tf.range(0, limit=real.shape[0])  # shape (batch_size)
    for dec_step, dist in enumerate(final_dists):
        # The indices of the target words. shape (batch_size)
        targets = real[:, dec_step]
        indices = tf.stack((batch_nums, targets), axis=1)  # shape (batch_size, 2)
        gold_probs = tf.gather_nd(dist, indices)  # shape (batch_size). prob of correct words on this step
        losses = -tf.math.log(gold_probs)
        loss_per_step.append(losses)
    # Apply dec_padding_mask and get loss
    _loss = _mask_and_avg(loss_per_step, padding_mask)
    return _loss

建立train-step

In [38]:
# @tf.function
def train_step(inp, targ, enc_hidden):
    # 初始化loss
    loss = 0
    with tf.GradientTape() as tape:
        # encoder实例化，同时也是初始化的过程
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        # 确定decoder的输入
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
        # 开始train的过程
        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            # 计算第一步输出后的loss值
            loss += loss_function(targ[:, t], predictions)
            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    # 计算一个训练样本的loss
    batch_loss = (loss / int(targ.shape[1]))
    # 反向梯度求导
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(batch_loss, variables) 
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss